In [19]:
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/Bioscienze/Progetto"

In [63]:
data = pd.read_csv(os.path.join(base_path,'AGM.txt'), sep="\t", dtype=object)
print('Numero di campioni: ', data.shape[0], '\nInformazioni: ', data.shape[1]-1)

Numero di campioni:  29960 
Informazioni:  660


In [12]:
data.head()

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
0,10317.000001000,Not provided,false,false,"Diagnosed by a medical professional (doctor, p...",60s,64.0,64,true,Daily,...,Not provided,Never,Regularly (3-5 times/week),Not provided,NaN,Remained stable,52,kilograms,Never,Occasionally (1-2 times/week)
1,10317.000001001,Not provided,false,false,Not provided,50s,53.0,53.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,false,false,Not provided,50s,53.0,53.0,true,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,false,false,Not provided,40s,44.0,44.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,false,false,Not provided,60s,66.0,66.0,false,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided


In [6]:
list(data.columns)

['sample_name',
 'acid_reflux',
 'acne_medication',
 'acne_medication_otc',
 'add_adhd',
 'age_cat',
 'age_corrected',
 'age_years',
 'alcohol_consumption',
 'alcohol_frequency',
 'alcohol_types',
 'alcohol_types_beercider',
 'alcohol_types_red_wine',
 'alcohol_types_sour_beers',
 'alcohol_types_spiritshard_alcohol',
 'alcohol_types_unspecified',
 'alcohol_types_white_wine',
 'allergic_to',
 'allergic_to_i_have_no_food_allergies_that_i_know_of',
 'allergic_to_other',
 'allergic_to_peanuts',
 'allergic_to_shellfish',
 'allergic_to_tree_nuts',
 'allergic_to_unspecified',
 'altitude',
 'alzheimers',
 'animal_age',
 'animal_free_text',
 'animal_gender',
 'animal_origin',
 'animal_type',
 'anonymized_name',
 'antibiotic_history',
 'appendix_removed',
 'artificial_sweeteners',
 'asd',
 'assigned_from_geo',
 'autoimmune',
 'birth_year',
 'bmi',
 'bmi_cat',
 'bmi_corrected',
 'body_habitat',
 'body_product',
 'body_site',
 'bowel_movement_frequency',
 'bowel_movement_quality',
 'breastmilk_for

In [76]:
data.specialized_diet_exclude_refined_sugars.value_counts()

Not provided      11341
No                 7138
false              1779
Yes                1438
true                367
Unspecified         187
Not applicable       10
Name: specialized_diet_exclude_refined_sugars, dtype: int64

In [77]:
data.specialized_diet_exclude_dairy.value_counts()

Not provided      11341
No                 7616
false              1958
Yes                 960
true                188
Unspecified         187
Not applicable       10
Name: specialized_diet_exclude_dairy, dtype: int64

In [78]:
data = data.loc[data.body_site == "UBERON:feces",:]

In [95]:
column_filt = ["sample_name",
               "age_years",
               "sex",
               "specialized_diet_exclude_dairy",
               "specialized_diet_exclude_refined_sugars",
               "body_site",
               "specialized_diet_i_do_not_eat_a_specialized_diet"]

In [96]:
data_f = data.loc[:,column_filt]

In [97]:
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "false","specialized_diet_i_do_not_eat_a_specialized_diet"] = "No"
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "true","specialized_diet_i_do_not_eat_a_specialized_diet"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_dairy == "false","specialized_diet_exclude_dairy"] = "No"
data_f.loc[data_f.specialized_diet_exclude_dairy == "true","specialized_diet_exclude_dairy"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "false","specialized_diet_exclude_refined_sugars"] = "No"
data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "true","specialized_diet_exclude_refined_sugars"] = "Yes"

In [98]:
data_diet = data_f.loc[(data_f.specialized_diet_exclude_dairy == "No") |
                        (data_f.specialized_diet_exclude_dairy == "Yes") |
                        (data_f.specialized_diet_exclude_refined_sugars == "No") |
                        (data_f.specialized_diet_exclude_refined_sugars == "Yes")|
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No") |
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "Yes"),:]

In [99]:
data_diet

,sample_name,age_years,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet
74,10317.000001166,47,female,No,No,UBERON:feces,Yes
93,10317.000001198,2,male,Yes,Yes,UBERON:feces,No
98,10317.000001210,75,female,No,No,UBERON:feces,No
104,10317.000001226,67,male,No,No,UBERON:feces,No
141,10317.000001300,50,female,No,No,UBERON:feces,No
...,...,...,...,...,...,...,...
26335,10317.000115687,5,male,No,No,UBERON:feces,Yes
26345,10317.000116371,64,male,No,No,UBERON:feces,Yes
26346,10317.000116374,69,female,No,No,UBERON:feces,Yes
26347,10317.000116375,34,male,No,No,UBERON:feces,Yes


In [112]:
pd.crosstab(data_diet.specialized_diet_exclude_refined_sugars,
            data_diet.specialized_diet_exclude_dairy, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_dairy,No,Yes,All
specialized_diet_exclude_refined_sugars,,,
No,7857,552,8409
Yes,1199,546,1745
All,9056,1098,10154


In [110]:
list(data_diet.sex.unique())

['female', 'male', 'Not provided', 'unspecified', 'other']

In [120]:
data_diet_f = data_diet[((data_diet['age_years'] != 'Unspecified') &
                      (data_diet['age_years'] != 'Not provided'))  &
                      ((data_diet.sex == 'female') |
                        (data_diet.sex == 'male'))]

In [121]:
data_diet_f.age_years = data_diet_f.age_years.astype(int)

In [123]:
data_diet_f = data_diet_f[(data_diet_f.age_years >24) & (data_diet_f.age_years < 56)]

In [127]:
np.sum((data_diet_f.specialized_diet_exclude_dairy == "Yes") & 
                            (data_diet_f.specialized_diet_exclude_refined_sugars == "No") &
                            (data_diet_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No")) 

285

In [124]:
data_diet_f.exclude_dairy = ((data_diet_f.specialized_diet_exclude_dairy == "Yes") & 
                            (data_diet_f.specialized_diet_exclude_refined_sugars == "No") &
                            (data_diet_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No")) 

data_diet_f.exclude_dairy = ((data_diet_f.specialized_diet_exclude_dairy == "Yes") & 
                            (data_diet_f.specialized_diet_exclude_refined_sugars == "No") &
                            (data_diet_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No")) 


,sample_name,age_years,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet
74,10317.000001166,47,female,No,No,UBERON:feces,Yes
141,10317.000001300,50,female,No,No,UBERON:feces,No
279,10317.000001586,41,female,No,No,UBERON:feces,No
555,10317.000002224,36,male,No,No,UBERON:feces,No
709,10317.000002595,36,female,No,Yes,UBERON:feces,No
...,...,...,...,...,...,...,...
26289,10317.000115212,43,male,No,No,UBERON:feces,Yes
26290,10317.000115214,43,male,No,No,UBERON:feces,Yes
26294,10317.000115221,36,male,No,No,UBERON:feces,Yes
26302,10317.000115230,27,male,No,Yes,UBERON:feces,Yes
